# Homework 3 - Places of the world
### Group #23 of students:
1. Viktoriia Vlasenko (viktoriiavvlasenko@gmail.com)
2. Chiara Sammarco (sammarco.1913440@studenti.uniroma1.it)
3. Nasim Sadeghi Khanshir (nasiimsadeghii@gmail.com)

### Research questions structure:
1. [RQ1] Data collection
2. [RQ2] Search Engine
3. [RQ3] Define a new score!
4. [RQ4] Visualizing the most relevant places
5. [RQ5] BONUS: More complex search engine
6. [RQ6] Command line question
7. [RQ7] Theoretical question 

## [RQ1] Data collection
### 1.1. Get the list of places
For retrieving data list of places we are collecting urls of topics of places.

In [1]:
# custom module with functions for assigment
import adm3
import os

In [6]:
# base url for places retrieving
root_url = "https://www.atlasobscura.com"

In [5]:
# for saving all files in current directory
print("File location using os.getcwd():", os.getcwd())

File location using os.getcwd(): C:\Users\vikto\Data_Science\ADM\ADM3


'Places.txt' contains url of places in format:
'/places/name-of-place'

In [7]:
f = open('Places.txt','a')
for i in tqdm(range(1,401)):
    l = f"{root_url}/places?page={i}&sort=likes_count"
    list_page = requests.get(l)
    list_soup = BeautifulSoup(list_page.text)
    list_places = [x.get('href') for x in list_soup.find_all('a', {'class':"content-card content-card-place"})]
    for x in list_places:
        f.write(x + "\n")
f.close()

100%|██████████| 400/400 [08:26<00:00,  1.27s/it]


### 1.2. Crawl places
For crawling places was written function saveHtmlToFolder().

In [8]:
help(adm3.saveHtmlToFolder)

Help on function saveHtmlToFolder in module adm3:

saveHtmlToFolder(folder_number, place_name, url)
    description:
        function that is used for saving html content to local directory
    
    params:
        - folder_name: number of page to save file in same directory;
        - place_name: name of place to name saved file;
        - url: url that was retrieved from 'Places.txt' file.
    output:
        logging info about saving files



##### There is a list of common reasons of interruptions:
1. ChunkedEncodingError: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))
2. non-stable internet connection: as result some files were downloaded, but its content was empty

In [7]:
urls = open('Places.txt', 'r')

link_counter = 0
for page in range(1, 401):
    print(f"CURRENT PAGE: {page}")
    page_urls = []
    for link in range(18):
        page_urls.append(urls.readline())
    readed = False
    while readed != True:
        read_statuses = []
        output = 1
        for link in page_urls:
            output = adm3.saveHtmlToFolder(page, link[:-1], root_url+link[:-1])
            read_statuses.append(output)
        if sum(read_statuses) == 0: 
            readed = True
    
urls.close()

CURRENT PAGE: 1
'https://www.atlasobscura.com/places/city-hall-station' is saved to 'C:\Users\vikto\Data_Science\ADM\ADM3\places\1\city-hall-station'
'https://www.atlasobscura.com/places/highgate-cemetery' is saved to 'C:\Users\vikto\Data_Science\ADM\ADM3\places\1\highgate-cemetery'
'https://www.atlasobscura.com/places/leadenhall-market' is saved to 'C:\Users\vikto\Data_Science\ADM\ADM3\places\1\leadenhall-market'
'https://www.atlasobscura.com/places/wave-organ' is saved to 'C:\Users\vikto\Data_Science\ADM\ADM3\places\1\wave-organ'
'https://www.atlasobscura.com/places/catacombes-de-paris' is saved to 'C:\Users\vikto\Data_Science\ADM\ADM3\places\1\catacombes-de-paris'
'https://www.atlasobscura.com/places/evolution-store' is saved to 'C:\Users\vikto\Data_Science\ADM\ADM3\places\1\evolution-store'
'https://www.atlasobscura.com/places/magowans-infinite-mirror-maze' is saved to 'C:\Users\vikto\Data_Science\ADM\ADM3\places\1\magowans-infinite-mirror-maze'
'https://www.atlasobscura.com/places

#### 1.3 Parse downloaded pages
For saving parsed data we created "place_i.tsv", that contains defined fields:
1. placeName: string
2. placeTags: list of strings
3. numPeopleVisited: int
4. numPeopleWant: int
5. placeDesc: string
6. placeShortDesc: string
7. placeNearby: list of strings
8. placeAddress: string
9. placeAlt: float
10. placeLong: float
11. placeEditors: list of strings
12. placePubDate: datatime
13. placeRelatedLists: list of strings
14. placeRelatedPlaces: list of strings
15. placeURL: string

In [9]:
# write titles
with open("place_i.tsv", "w", encoding="utf8") as f_tsv_titles:
    titles = " \t ".join(["placeName", 
                 "placeTags", 
                 "numPeopleVisited", 
                 "numPeopleWant", 
                 "placeDesc", 
                 "placeShortDesc", 
                 "placeNearby", 
                 "placeAddress", 
                 "placeAlt", 
                 "placeLong", 
                 "placeEditors", 
                 "placePubDate",
                 "placeRelatedLists",
                 "placeRelatedPlaces",
                 "placeURL"])
    f_tsv_titles.write(titles+"\n")  

For parsing places was written function parse().

In [2]:
help(adm3.parse)

Help on function parse in module adm3:

parse(filename, page)
    description:
        parse file to Place object with defined fields
    params:
        - filename: name of file for parsing
        - page: current page
    return:
        Place object with parsed data



For data moving was created class Place. Each object contains data of parsed html file.

In [5]:
import inspect
inspect.getmembers(adm3.Place, lambda a:not(inspect.isroutine(a)))

[('__class__', type),
 ('__dict__',
  mappingproxy({'__module__': 'adm3',
                '__init__': <function adm3.Place.__init__(self, placeName, placeTags, numPeopleVisited, numPeopleWant, placeDesc, placeShortDesc, placeNearby, placeAddress, placeAlt, placeLong, placeEditors, placePubDate, placeRelatedLists, placeRelatedPlaces, placeURL)>,
                'to_string': <function adm3.Place.to_string(self)>,
                '__dict__': <attribute '__dict__' of 'Place' objects>,
                '__weakref__': <attribute '__weakref__' of 'Place' objects>,
                '__doc__': None})),
 ('__doc__', None),
 ('__module__', 'adm3'),
 ('__weakref__', <attribute '__weakref__' of 'Place' objects>)]

For saving Place object to 'place_i.tsv' was created function save_place_single_data().

In [3]:
help(adm3.save_place_single_data)

Help on function save_place_single_data in module adm3:

save_place_single_data(place, fp)
    description:
        saves Place object to tsv file
    params:
        - place: Place object that has .to_string method to cast its data to string
        - fp: 'place_i.tsv' file that is opened for appending (writing) files



In [8]:
current_path = os.getcwd()
number_of_parsed_files = 0
number_of_skipped_files = 0
with open("place_i.tsv", "a", encoding="utf8") as fp:  
    for page in range(1, 401):
        print(f"CURRENT PAGE: {page}")
        current_folder = os.path.join(current_path, "places", str(page))
        filenames = os.listdir(current_folder)
        for filename in filenames:
            # html file is parsed to Place object
            parsed_place = adm3.parse(filename, page)
            if parsed_place == None: 
                number_of_skipped_files += 1
                print(filename)
            else: number_of_parsed_files += 1
            # save Place object to "place_i.tsv"
            adm3.save_place_single_data(parsed_place, fp)
print(number_of_parsed_files, number_of_skipped_files)

CURRENT PAGE: 1
placeDesc
placeDesc
placeDesc
placeDesc
placeDesc
CURRENT PAGE: 2
placeDesc
placeDesc
placeDesc
placeDesc
CURRENT PAGE: 3
placeDesc
placeDesc
placeDesc
CURRENT PAGE: 4
placeDesc
placeDesc
placeDesc
placeDesc
placeDesc
placeDesc
CURRENT PAGE: 5
placeDesc
placeDesc
placeDesc
placeDesc
CURRENT PAGE: 6
placeDesc
placeDesc
placeDesc
placeDesc
placeDesc
CURRENT PAGE: 7
placeDesc
placeDesc
placeDesc
placeDesc
placeDesc
placeDesc
CURRENT PAGE: 8
placeDesc
placeDesc
placeDesc
placeDesc
placeDesc
CURRENT PAGE: 9
placeDesc
placeDesc
placeDesc
CURRENT PAGE: 10
placeDesc
placeDesc
CURRENT PAGE: 11
placeDesc
placeDesc
placeDesc
placeDesc
placeDesc
CURRENT PAGE: 12
placeDesc
placeDesc
placeDesc
placeDesc
placeDesc
CURRENT PAGE: 13
placeDesc
placeDesc
placeDesc
placeDesc
placeDesc
placeDesc
CURRENT PAGE: 14
placeDesc
placeDesc
placeDesc
placeDesc
CURRENT PAGE: 15
placeDesc
placeDesc
CURRENT PAGE: 16
placeDesc
placeDesc
placeDesc
CURRENT PAGE: 17
placeDesc
placeDesc
placeDesc
placeDesc


##### Conslusions about created dataset:
1. By viewing the logs of parsing (if it catches an exception while parsing defined field, function will skip that filed for writing to the dataset file) we can say that some files don't have placeDesc.
2. Also there was some dataloss while parsing in case of non-defined information that were searched in files.
3. The main unexpected result: after proccess of parsing crawled files the script appending +500 rows with titles.

So for using created dataset we have to 'clean' it.

In [9]:
import pandas as pd
df = pd.read_csv('place_i.tsv', sep=' \t ', engine='python')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7200 entries, 0 to 7199
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   placeName           7200 non-null   object 
 1   placeTags           6608 non-null   object 
 2   numPeopleVisited    6608 non-null   float64
 3   numPeopleWant       6608 non-null   float64
 4   placeDesc           4419 non-null   object 
 5   placeShortDesc      7200 non-null   object 
 6   placeNearby         6608 non-null   object 
 7   placeAddress        6608 non-null   object 
 8   placeAlt            6608 non-null   float64
 9   placeLong           7200 non-null   object 
 10  placeEditors        6608 non-null   object 
 11  placePubDate        7199 non-null   object 
 12  placeRelatedLists   7200 non-null   object 
 13  placeRelatedPlaces  7200 non-null   object 
 14  placeURL            6608 non-null   object 
dtypes: float64(3), object(12)
memory usage: 843.9+ KB


In [15]:
df = df[:7201]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7200 entries, 0 to 7199
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   placeName           7200 non-null   object 
 1   placeTags           6608 non-null   object 
 2   numPeopleVisited    6608 non-null   float64
 3   numPeopleWant       6608 non-null   float64
 4   placeDesc           4419 non-null   object 
 5   placeShortDesc      7200 non-null   object 
 6   placeNearby         6608 non-null   object 
 7   placeAddress        6608 non-null   object 
 8   placeAlt            6608 non-null   float64
 9   placeLong           7200 non-null   object 
 10  placeEditors        6608 non-null   object 
 11  placePubDate        7199 non-null   object 
 12  placeRelatedLists   7200 non-null   object 
 13  placeRelatedPlaces  7200 non-null   object 
 14  placeURL            6608 non-null   object 
dtypes: float64(3), object(12)
memory usage: 843.9+ KB


## [RQ6] Command line question

In [2]:
%%cmd
bash
dos2unix ./CommandLine.sh
./CommandLine.sh

Microsoft Windows [Version 10.0.22000.1219]
(c) Microsoft Corporation. All rights reserved.

(base) C:\Users\vikto\Data_Science\ADM\ADM3>bash


dos2unix: converting file ./CommandLine.sh to Unix format...


Country: Italy
i. Number of places can be found in Italy:
191
ii. Number of people, on average, have visited the places in Italy:
400
iii. Number of people in total want to visit the Italy:
172080
----------------------------------------------------------------
Country: Spain
i. Number of places can be found in Spain:
83
ii. Number of people, on average, have visited the places in Spain:
483
iii. Number of people in total want to visit the Spain:
67329
----------------------------------------------------------------
Country: France
i. Number of places can be found in France:
190
ii. Number of people, on average, have visited the places in France:
448
iii. Number of people in total want to visit the France:
196545
----------------------------------------------------------------
Country: England
i. Number of places can be found in England:
341
ii. Number of people, on average, have visited the places in England:
491
iii. Number of people in total want to visit the England:
375982
-------